In [17]:
# hide
!pip install reed

In [18]:
# hide
import re
import json
from html.parser import HTMLParser

import numpy as np

In [19]:
# hide
class MyHTMLParser(HTMLParser):

    def __init__(self):
        self.string = ''
        super().__init__()

    def handle_data(self, data):
        self.string = self.string + ' ' + data
        return (data)

    def return_data(self):
        return self.string.strip().replace('  ', ' ')

In [20]:
# hide
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'creds.json'", fields="files(id)").execute()
creds = results.get('files', [])
results = drive_service.files().list(
        q="name = 'job_data.csv'", fields="files(id)").execute()
data = results.get('files', [])

filename = "/content/.reed/creds.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=creds[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

filename = "/content/data/job_data.csv"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=data[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.
Download 100%.


In [21]:
# hide
filename = "/content/.reed/creds.json"
with open(filename, 'r') as f:
    creds = json.load(f)
    
api_key=creds['API_KEY']

In [22]:
api_key = '800d8a98-ca95-4acb-b6ab-2d88d46d85b9'

## Intro

Visualisation is an essential part of any data role. Whether we are delivering regular reports to decision-makers, presenting the results of a regression model to a room of stakeholders, or creating a real-time dashboard for the wider business, we are using data to tell a story, and visuals bring these stories to life.

In this post we are going to explore a number of ways to visualise text data in the context of analysing job posts for three data roles: data scientists, data engineers and data analysts. These roles share certain commonalities but are fundamentally different; will the content of the job descriptions for these roles reflect the differences between them? And how can we visualise these differences?

## Getting the data

First we need to collect some job descriptions to analyse. There are plenty of job search APIs out there to choose from and we also have the option to scrape directly from job websites. I ended up using the Reed developer API because of the simple process of signing up for credentials. I wrote a python wrapper for the API which I will be using to extract a collection of relevant job posts; you can find it [here](https://pypi.org/project/reed/).

It's a simple process to fetch the jobs matching a keywords search query. The response is a list of json objects which in this case are identical to python dictionaries so we can easily place the data in a pandas dataframe. 

In [23]:
import pandas as pd
from reed import ReedClient

client = ReedClient(api_key=api_key)
search_params = {
    'keywords': 'data+scientist|data+engineer|data+analyst', 
    'resultsToTake': 600
}
response = client.search(**search_params)
df = pd.DataFrame(response)
df.shape

(524, 15)

Our search returned 538 jobs in total; let's clean up the job titles and see what we have.

In [24]:
def clean_title(title_str): 
    return title_str.lower().strip().replace('  ', ' ')

df['jobTitle'] = [clean_title(x) for x in df.jobTitle]
df.groupby('jobTitle').size().sort_values(ascending=False).head(10)

jobTitle
data analyst                   62
data engineer                  46
data scientist                 43
senior data scientist          19
senior data engineer           11
big data engineer               5
lead data scientist             5
lead data engineer              4
data scientist - manchester     4
graduate data analyst           4
dtype: int64

We're going to remove all the posts which don't have one of the three most common titles; what we're left with should be broadly similar in terms of representing mid-level roles within each category. 

In [25]:
accepted_titles = ['data scientist', 'data engineer', 'data analyst']
df = df[[x in accepted_titles for x in df.jobTitle]].set_index('jobId')

The job search API gives a truncated version of the job descriptions so if we want the complete text we'll have to take each individual job id and pull down the details using the job details function one-by-one. To make this process easier I created a dictionary with the three job titles as keys and the items consisting of the list of ids corresponding to each title.

In [26]:
groups = df.groupby('jobTitle').groups
groups.keys()

dict_keys(['data analyst', 'data engineer', 'data scientist'])

Now we can loop over the dictionary pulling down the description from each job in turn. There's another complication in that the job descriptions are returned as html documents. We're only interested in the text data so we're going to have the parse the html to extract the information we want. We can wrap this process inside a function which we call for each id in our dictionary. 

In [27]:
def get_job_desc(job_type, job_id, client, html_parser):
    desc_html = client.job_details(job_id)['jobDescription']
    parser.feed(desc_html)
    desc_str = parser.return_data()
    # reset parser string
    parser.string = ''
    return dict(job=job_type, job_id=job_id, desc=desc_str)
    
parser = MyHTMLParser()
job_descriptions = []
for title in groups:
    for id in groups[title]:
        job_desc_dict = get_job_desc(title, id, client, parser)
        job_descriptions.append(job_desc_dict)
        
df = pd.DataFrame(job_descriptions)
df.head()

,job,job_id,desc
0,data analyst,40532763,"Data Analyst £35,000 - £40,000 + Benefits Lo..."
1,data analyst,40500203,A well established Telecommunications company ...
2,data analyst,40500186,A well established Telecommunications company ...
3,data analyst,40531962,"My client, a multinational, speciality insuran..."
4,data analyst,40531668,"My client, a charitable organisation with stro..."


Now we're (nearly) ready to get into the good stuff. We need to do a bit more tidying before we get started on the visualisations. In order to visualise the content of the job descriptions we need a numerical representation of the text. One way to do this is with a bag-of-words approach, which consists of separating the text in our documents into tokens and counting the appearances of each token in each document. Sklearn can do this for us with its CountVectorizer class.

In [28]:
# hide
filename = "/content/data/job_data.csv"
df = pd.read_csv(filename, index_col=0)

In [29]:
# hide
df = df[~df.duplicated('desc')]

cicd_pat = "([Cc][Ii]/[Cc][Dd])"
phd_pat = "[Pp][Hh].?[Dd]"
r_pat = '(\sR\W)'

df['desc'] = (df.desc.str.replace(',000', 'k')
             .str.replace('\xa0', ' ')
             .str.replace(phd_pat, 'PHD')
             .str.replace(r_pat, ' RStudio ')
             .str.replace(cicd_pat, ' CICD')
             .str.replace('Modis', ''))

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')
text_data = np.array(df['desc'])
count_matrix = count_vectorizer.fit_transform(text_data)
count_matrix_df = pd.DataFrame(count_matrix.toarray(), columns=count_vectorizer.get_feature_names())
count_matrix_df

,00,000,025,04,06,075,08,10,100,100k,1070906,11,12,13,1376154,14,15,16,17,18,184,19,1997,1x,20,200,2003,2012,2018,2019,2020,20th,210,22nd,23,24,240,24k,24x7,25,...,won,word,words,work,worked,workers,workflow,workflows,workforce,working,workload,workloads,workplace,works,workshops,world,worldwide,worry,worth,wrangling,wrexham,wrigley,write,writers,writing,written,wrong,wtl,ww,www,x4,xml,yagni,yarn,year,years,yields,yoga,yorkshire,zachman
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,4,1,1,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


When we fit the CountVectorizer transform to an array containing each job description what we get is a document-term matrix: the rows are the job descriptions and the columns are all the words which were found in the collection of documents. The contents of the matrix are the word frequencies. How can we use this implementation to visualise the three different roles? A simple option is to look at the most frequent words across the three classes: if we group the documents into the three roles, sum the word frequencies and sort them in descending order we'll see which words appear the most in each. I've used plotly to create plots of the most frequent words in each role.

In [31]:
# hide_input
from plotly.subplots import make_subplots
import plotly.graph_objects as go

count_matrix_df['job'] = df.job
counts_by_job = (count_matrix_df.groupby('job')
                 .sum()
                 .transpose()
                 .reset_index()
                 .rename(columns={'index': 'word'}))

da_top = counts_by_job[['word', 'data analyst']].sort_values('data analyst', ascending=False)[:20]
de_top = counts_by_job[['word', 'data engineer']].sort_values('data engineer', ascending=False)[:20]
ds_top = counts_by_job[['word', 'data scientist']].sort_values('data scientist', ascending=False)[:20]

colours_dict = {'data analyst': '#636efa', 'data scientist': '#00cc96', 'data engineer': '#EF553B'}

fig = make_subplots(rows=1, 
                    cols=3, 
                    subplot_titles=("DS top words", "DE top words", "DA top words"),
                    x_title='frequency'
                )

fig.add_trace(go.Bar(
    y=ds_top['word'],
    x=ds_top['data scientist'],
    orientation='h',
    name='ds',
    marker=dict(color=colours_dict['data scientist'])),
    row=1, col=1
)

fig.add_trace(go.Bar(
    y=de_top['word'],
    x=de_top['data engineer'],
    orientation='h',
    name='de',
    marker=dict(color=colours_dict['data engineer'])),
    row=1, col=2
)

fig.add_trace(go.Bar(
    y=da_top['word'],
    x=da_top['data analyst'],
    orientation='h',
    name='da',
    marker=dict(color=colours_dict['data analyst'])),
    row=1, col=3
)

fig.update_layout(
    yaxis1=dict(autorange="reversed"),
    yaxis2=dict(autorange="reversed"),
    yaxis3=dict(autorange="reversed"),
    height=600, width=1000,
    showlegend=False
)
fig.show()

Not bad! We can spot a few themes such as the emphasis on machine learning in DS job descriptions while SQL and reporting feature prominently in DA posts. The plot corresponding to Data engineer roles is not as insightful. The appearance of Python, SQL and Azure are promising, but no sign of ETL pipelines and a lower prevalance of cloud-based systems than we'd expect. And why does scientist turn up so much? Across the three plots we can also see another problem with this simple word frequency approach: the most insightful terms are diluted by a saturation of words which are common across the three roles and thus essentially useless, such as 'data', 'experience', 'role' and 'team'. 

We need a metric which highly ranks frequent words but suppresses words which appear in lots of the documents in our corpus. Luckily for us a metric exists which does exactly as we require. TFIDF or term-frequency inverse-document frequency takes the term frequency of a word in a document and multiplies it by the inverse document freqency of that word, essentially damping those words which appear across a large proportion of the documents. Again sklearn has our back and can perform this process for us.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
max_df = 0.6
min_df = 2
vectorizer  = TfidfVectorizer(stop_words='english', max_df=max_df, min_df=min_df)
text = np.array(df['desc'])
tfidf_matrix = vectorizer.fit_transform(text)
tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names())
tfidf

,00,08,10,100,1070906,12,14,16,17,19,1997,20,2003,2018,2019,2020,20th,240,24k,25,25k,26,28,30,300,30k,31k,32k,35k,40,400,40k,450,45k,500,50k,55k,60,60k,65k,...,warrington,watch,watching,way,ways,web,website,week,weeks,welcome,welcomes,wellbeing,west,whilst,wide,wider,willing,willingness,winning,won,word,worked,workers,workflow,workflows,workforce,workload,workloads,workplace,works,world,worldwide,wrangling,wrexham,write,writing,written,x4,year,years
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.151358,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.168567,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.126809,0.0,0.086106,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.135622,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.257828,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.168772,0.096335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.07677,0.000000,0.0,0.0,0.0,0.0,0.000000,0.071167,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.102478,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.196656,0.0,0.105206,0.0,0.000000,0.110827,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.102739,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.100455,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.059399,0.0,0.000000,0.0,0.062064,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.054997,0.0,0.0,0.0,0.00000,0.079857,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.076462,0.0,0.073592,0.060684,0.0,0.0,0.0,0.0,0.0,0.000000,0.068912,0.053134,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.044717
129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069856,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.084834,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.075804,0.0,0.0,0.0,0.0,0.000000,0.000000,0.061867,0.0,0.0,0.063552,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0

As before the TfidfVectorizer returns a term-document matrix, but instead of consisting of word frequencies we have tfidf values for each term-document pair. The parameters 'min_df' and 'max_df' are constraints on the document frequency of the words in our vocabulary. We have used a 'min_df' of 2, so any words that do not appear in at least two documents are thrown out. 'Max_df' is a float which corresponds to a proportion of the documents, so here if any words appear in more than 60 percent of the documents they too are discarded. In the same way as before, we can sum the tfidf values within the three classes and plot the highest aggregate values to hopefully extract the most important terms to characterise the three job types.

In [33]:
# hide_input
tfidf_with_job = tfidf.copy()
tfidf_with_job['job_type'] = df.job
counts_by_job = (tfidf_with_job.groupby('job_type')
                 .sum()
                 .transpose()
                 .reset_index()
                 .rename(columns={'index': 'word'}))

da_top = counts_by_job[['word', 'data analyst']].sort_values('data analyst', ascending=False)[:20]
de_top = counts_by_job[['word', 'data engineer']].sort_values('data engineer', ascending=False)[:20]
ds_top = counts_by_job[['word', 'data scientist']].sort_values('data scientist', ascending=False)[:20]

colours_dict = {'data analyst': '#636efa', 'data scientist': '#00cc96', 'data engineer': '#EF553B'}

fig = make_subplots(rows=1, 
                    cols=3, 
                    subplot_titles=("DS top words", "DE top words", "DA top words"),
                    x_title='tfidf score'
                )

fig.add_trace(go.Bar(
    y=ds_top['word'],
    x=ds_top['data scientist'],
    orientation='h',
    marker=dict(color=colours_dict['data scientist'])),
    row=1, col=1
)

fig.add_trace(go.Bar(
    y=de_top['word'],
    x=de_top['data engineer'],
    orientation='h',
    marker=dict(color=colours_dict['data engineer'])),
    row=1, col=2
)

fig.add_trace(go.Bar(
    y=da_top['word'],
    x=da_top['data analyst'],
    orientation='h',
    marker=dict(color=colours_dict['data analyst'])),
    row=1, col=3
)

fig.update_layout(
    yaxis1=dict(autorange="reversed"),
    yaxis2=dict(autorange="reversed"),
    yaxis3=dict(autorange="reversed"),
    height=600, width=1050,
    showlegend=False
)
fig.show()

This is a big improvement over our first attempt. There are far fewer generic terms because of the introduction of the inverse-document weighting and the document frequency constraints. The data engineering terms are also closer to our expectations than they were before. 

What are the key insights we can take from these plots? For data scientists machine learning is the most distinguishing feature. R and Python both appear with the latter ranked slightly higher, at least in this sample. We can also see that masters degrees and doctorates are important in data scientist posts but not in the other two roles. This makes a certain degree of sense as data science typically relies on a more theoretical foundation than the other two fields. While data engineering is a technical field, it is centered on a deep understanding of specific technologies, an understanding which is more typically gained from extensive working experience than time spent in academia.

Speaking of data engineering, our picture of the field is clearer from this version of the plot. Cloud technologies and databases dominate and python seems to be the scripting language of choice within this sample of posts. The impression given by this visual is that tools and technologies are to data engineering what algorithms and models are to data science. 

Finally, data analyst positions appear to be characterised by aptitudes and skills than by specific technologies or technical backgrounds. SQL and excel feature heavily but apart from these tools the key terms seem to describe an general analytical mindset and an ability to support other business functions through reporting and analysis.

## Plotting job descriptions via dimensionality reduction

For those of you that only clicked on this post because unsupervised learning was in the title, this section is for you. Using TF-IDF we have created a 2038-dimensional representation of the job descriptions in our sample. As it is a bit of a struggle to visualise anything in more than three dimensions, it is common practice to manipulate high-dimensional data into a two or three dimensional space so that it can be visualised and structural patterns in the data can be more easily found. In this case we are going to use Principal Component Analysis, or PCA for short, to project our data onto the two dimensions which retain the highest variability. 

In [34]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X = pca.fit_transform(tfidf)
X_df = pd.DataFrame(X)
X_df['job'] = df.job.values
X_df = X_df.rename(columns={0: 'pc_0', 1: 'pc_1'})
X_df.sample(5)

,pc_0,pc_1,job
3,-0.141825,-0.070398,data analyst
125,0.419590,-0.174660,data scientist
82,0.057180,0.211558,data engineer
87,0.009647,0.116863,data engineer
97,0.283853,-0.002391,data scientist


In [35]:
pca.explained_variance_

array([0.0300115 , 0.02438873])

Using only a two dimensional representation of the tfidf vectors we retain just over half of the variance in our data. The following is a scatterplot of the first two principle component scores; each point is a job description. 

In [36]:
import plotly.express as px
fig = px.scatter(X_df, x="pc_0", y="pc_1", color="job", hover_data=["pc_0", "pc_1", X_df.index])
fig.update_layout(title='first two PC scores of tfidf matrix')
fig.show()

Remarkably, the three different job titles are almost perfectly separated into three clusters of data points in this two dimensional representation. This is significant because the PCA algorithm has no idea that our dataset contains three distinct classes, but by simply projecting the data onto the coordinate axes retaining the most variance the algorithm has managed to almost perfectly separate them. We can go one step further and demonstrate that the three classes are well-separated by applying k-means clustering to the principle component scores. The result is the following.

In [37]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
centers = kmeans.cluster_centers_

In [38]:
# hide_input
fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.15,
                   subplot_titles=("actual job titles","k means clustered job titles"))

fig1 = px.scatter(X_df, x="pc_0", y="pc_1", color="job", hover_data=["pc_0", "pc_1", X_df.index])
trace1, trace2, trace3 = fig1['data'][0], fig1['data'][1], fig1['data'][2]
trace1['name'] = 'data analyst'
trace2['name'] = 'data engineer'
trace3['name'] = 'data scientist'

plotly_colours = px.colors.qualitative.Plotly
c = [plotly_colours[x] for x in y_kmeans]

fig2 = px.scatter(X_df, x='pc_0', y='pc_1', color=c, hover_data=['job', 'pc_0', 'pc_1', X_df.index])
trace4, trace5, trace6 = fig2['data'][0], fig2['data'][1], fig2['data'][2]
trace4['name'] = 'data analyst'
trace5['name'] = 'data engineer'
trace6['name'] = 'data scientist'

temp_str = '<br>job=%{customdata[0]}' + \
            '<br>pc_0=%{customdata[1]}' + \
            '<br>pc_1=%{customdata[2]}' + \
            '<br>index=%{customdata[3]}' + \
            '<extra></extra>'

trace4['hovertemplate'] = temp_str
trace5['hovertemplate'] = temp_str
trace6['hovertemplate'] = temp_str

trace4['legendgroup'] = 'job=data analyst'
trace4['showlegend'] = False
trace5['legendgroup'] = 'job=data engineer'
trace5['showlegend'] = False
trace6['legendgroup'] = 'job=data scientist'
trace6['showlegend'] = False

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)
fig.add_trace(trace3, row=1, col=1)
fig.add_trace(trace4, row=2, col=1)
fig.add_trace(trace5, row=2, col=1)
fig.add_trace(trace6, row=2, col=1)

fig.add_trace(go.Scatter(
    x=centers[:, 0],
    y=centers[:, 1],
    name='cluster centers',
    mode='markers',
    marker=dict(size=[15, 15, 15],
                color=[2, 2, 2],
                opacity=0.7)),
    row=2, col=1
)

# fig['data'][6]['showlegend'] = False

fig.update_layout(height=800, legend=dict(x=1, y=0.5))
fig.layout['xaxis']['title'] = 'pc_0'
fig.layout['xaxis2']['title'] = 'pc_0'
fig.layout['yaxis']['title'] = 'pc_1'
fig.layout['yaxis2']['title'] = 'pc_1'
fig.show()

Let's recap what we've achieved here. We started with a collection of raw text documents, each one a description of a job from one of three titles. Using term-frequency inverse-document-freqency we converted each description into a fixed length vector. Then using PCA we projected our new 2038-dimensional data matrix onto the two coordinate axes of highest variability so we could produce a scatterplot of the data. The result was a clear visual separation into three clusters corresponding to the three job titles, which we have now shown are nearly perfectly recreated using a simple clustering algorithm.

The cool part about this is not that we have perfectly classified each tfidf vector with its job title. There are a whole host of supervised approaches that would have been more effective if that were our goal: we would certainly include more principle components to preserve more information and most classification algorithms would perform better than the simple clustering we have used here. No, the interesting part is thinking about how these simple unsupervised methods can be applied to problems where we *don't* have access to labelled data as we did here. In text problems where we are unsure of the existence of distinct clusters we can go through the same steps, cluster the data and then use the tfidf vectors as we did before to look for patterns in the most important words in each cluster.

Clustering is far from a cure-all concept that works magic in every use-case, but it can be a fruitful area of investigation across a wide range of problems. We have shown how it can be effective in a simple, small-sample case but I hope that you can use some of these ideas in your own problems.